<a href="https://colab.research.google.com/github/dtabuena/Ephys_DataSets/blob/main/Analyze_BMX_HEK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)



!pip install openpyxl
!pip install XlsxWriter


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',

            'Ephys_wrapper.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f




Cloning into 'EphysLib'...
remote: Enumerating objects: 1141, done.
remote: Counting objects: 100% (519/519), done.
remote: Compressing objects: 100% (250/250), done.
remote: Total 1141 (delta 378), reused 357 (delta 269), pack-reused 622
Receiving objects: 100% (1141/1141), 17.41 MiB | 11.24 MiB/s, done.
Resolving deltas: 100% (754/754), done.


In [125]:
'https://www.dropbox.com/scl/fi/dnvbpjq6jhv9mvn23wrij/HEK-KCNT2.xlsx?rlkey=idsmjb0cc5fow087p190h0n6f&dl=1'

'https://www.dropbox.com/scl/fi/dnvbpjq6jhv9mvn23wrij/HEK-KCNT2.xlsx?rlkey=idsmjb0cc5fow087p190h0n6f&dl=1'

In [126]:
import urllib
file_name,_ = urllib.request.urlretrieve('https://www.dropbox.com/scl/fi/1yg80je83aqr68berui86/Before_After.csv?rlkey=k9pacq5t9u9icu1nkcnrhoad6&dl=1','before_after.csv')
doses_df = pd.read_csv(file_name)
display(doses_df.head(15))

,Cell,Before IV,During,After IV
0,2023x12x13_SLICK_c001_HEK,2023x12x13_SLICK_c001_HEK_0002,3,2023x12x13_SLICK_c001_HEK_0004
1,2023x12x13_SLICK_c002_HEK,2023x12x13_SLICK_c002_HEK_0004,6,2023x12x13_SLICK_c002_HEK_0008
2,2023x12x14_SLICK_c001_HEK,2023x12x14_SLICK_c001_HEK_0000,2,2023x12x14_SLICK_c001_HEK_0003
3,2023x12x15_SLICK_c001_HEK,2023x12x15_SLICK_c001_HEK_0003,4,2023x12x15_SLICK_c001_HEK_0005
4,2023x12x15_SLICK_c002_HEK,2023x12x15_SLICK_c002_HEK_0002,3,2023x12x15_SLICK_c002_HEK_0004
5,2023x12x15_SLICK_c003_HEK,2023x12x15_SLICK_c003_HEK_0001,2,2023x12x15_SLICK_c003_HEK_0003
6,2023x12x15_SLICK_c005_HEK,2023x12x15_SLICK_c005_HEK_0002,3,2023x12x15_SLICK_c005_HEK_0005
7,2023x12x15_SLICK_c006_HEK,2023x12x15_SLICK_c006_HEK_0002,4,2023x12x15_SLICK_c006_HEK_0007
8,2023x12x15_SLICK_c007_HEK,2023x12x15_SLICK_c007_HEK_0001,3,2023x12x15_SLICK_c007_HEK_0005
9,2023x12x15_SLICK_c008_HEK,2023x12x15_SLICK_c008_HEK_0001,3,2023x12x15_SLICK_c008_HEK_0004


In [127]:
dataset = {'data_name': 'BMX',
           'data_source': "https://www.dropbox.com/scl/fo/2ps1mdb88490cgol95l3u/h?rlkey=trp33s4fhrl4pyg4h4k0jecwh&dl=1",
           'file_naming_scheme': ['Rec_date','GenoType','Cell_num','Cell_Type'],
           }
##2023x12x13_SLICK_c001_HEK_0001
data_name = dataset['data_name']
data_source = dataset['data_source']
file_naming_scheme = dataset['file_naming_scheme']

''' Gather and Catalog Source Data'''
file_loc = get_drobox_folder_url(data_source, 'my_ephys_data_' + data_name)
clear_output(wait=False)
abf_recordings_df, protocol_set = catalogue_recs(file_loc,file_naming_scheme)
print(protocol_set)

['K_total_Run', 'VC - MemTest-10ms-160ms', 'K_total']


In [190]:
from sklearn import linear_model

def measure_slick(abf,duration_ms = 10,to_plot=True):
    duration_ms = 10
    duration_indx = int(duration_ms/1000*abf.sampleRate)
    measure_epoch = get_epoch(abf,epoch=3)
    stop_idx = measure_epoch['index']["stop"]
    start_idx = stop_idx - duration_indx
    if 2 in abf.channelList: i_chan = 2
    else: i_chan = 0
    I,V,T = analyze_currents(abf,start_idx,stop_idx,i_chan=i_chan)
    return {'Ik':I,'Vm':V,'SweepTimes':T}

def get_epoch(abf,epoch=3):
    results={"epoch":epoch}
    results={"index":{"start":abf.sweepEpochs.p1s[epoch],
                      "stop":abf.sweepEpochs.p1s[epoch+1]}}
    return results

def analyze_currents(abf,start_idx,stop_idx,analysis_function=np.median,i_chan=0,v_chan=1):
    I = list()
    V = list()
    for s in abf.sweepList:
        abf.setSweep(s,channel=i_chan)
        I.append(analysis_function(abf.sweepY[start_idx:stop_idx]))
        abf.setSweep(s,channel=v_chan)
        V.append(analysis_function(abf.sweepY[start_idx:stop_idx]))
    V = [int(10*np.round(v/10,0)) for v in V]
    return I,V,list(abf.sweepTimesSec)


def linear_leak_correction(sweep_results,v_range=(-55,-10)):
    Vm = np.array(sweep_results['Vm'])[:,np.newaxis]
    I = np.array(sweep_results['Ik'])[:,np.newaxis]
    fit_ind = [i for i,v in enumerate(Vm) if v >= np.min(v_range) and v<= np.max(v_range) ]
    if len(fit_ind)<2:
        return dict()
    x = Vm[fit_ind]
    y = I[fit_ind]
    leak_model = linear_model.LinearRegression().fit(x,y)
    leak_pred  = leak_model.predict(Vm)
    corrected_I = I - leak_pred
    corrected_I = [float(c) for c in corrected_I]
    # plt.scatter(Vm,I,c='k')
    # plt.plot(Vm,leak_pred,'k')
    # plt.scatter(Vm,corrected_I,c='r')
    sweep_results['Ik_corr'] =corrected_I
    return sweep_results


In [191]:
to_plot = True
results_list = list()
for rec in abf_recordings_df.index:
    abf = pyabf.ABF(rec)
    sweep_results = measure_slick(abf,duration_ms = 10,to_plot=True)
    if abf_recordings_df.loc[rec,'protocol'] == 'K_total':
        abf = pyabf.ABF(rec)
        sweep_results = linear_leak_correction(sweep_results)
    results_list.append(sweep_results)
abf_recordings_df['K_Currents']=results_list
display(abf_recordings_df.head())

,Recording_name,cell_id,Rec_date,GenoType,Cell_num,Cell_Type,protocol,abf_timestamp,channelList,K_Currents
file_name,,,,,,,,,,
my_ephys_data_BMX/2023x12x13/2023x12x13_SLICK_c001_HEK_0001.abf,2023x12x13_SLICK_c001_HEK_0001.abf,2023x12x13_SLICK_c001_HEK,2023x12x13,SLICK,c001,HEK,K_total,2023-12-13T14:33:32.563,"[0, 1]","{'Ik': [-403.74756, -351.25732, -330.2002, -26..."
my_ephys_data_BMX/2023x12x13/2023x12x13_SLICK_c001_HEK_0002.abf,2023x12x13_SLICK_c001_HEK_0002.abf,2023x12x13_SLICK_c001_HEK,2023x12x13,SLICK,c001,HEK,K_total,2023-12-13T14:34:36.753,"[0, 1]","{'Ik': [-494.0796, -458.6792, -399.4751, -369...."
my_ephys_data_BMX/2023x12x13/2023x12x13_SLICK_c001_HEK_0003.abf,2023x12x13_SLICK_c001_HEK_0003.abf,2023x12x13_SLICK_c001_HEK,2023x12x13,SLICK,c001,HEK,K_total_Run,2023-12-13T14:38:07.605,"[0, 1]","{'Ik': [-102.23389, -116.57715, -114.13574, -1..."
my_ephys_data_BMX/2023x12x13/2023x12x13_SLICK_c001_HEK_0004.abf,2023x12x13_SLICK_c001_HEK_0004.abf,2023x12x13_SLICK_c001_HEK,2023x12x13,SLICK,c001,HEK,K_total,2023-12-13T14:43:13.525,"[0, 1]","{'Ik': [-772.7051, -707.70264, -603.02734, -51..."
my_ephys_data_BMX/2023x12x13/2023x12x13_SLICK_c001_HEK_0005.abf,2023x12x13_SLICK_c001_HEK_0005.abf,2023x12x13_SLICK_c001_HEK,2023x12x13,SLICK,c001,HEK,K_total,2023-12-13T14:43:45.072,"[0, 1]","{'Ik': [-763.5498, -706.7871, -603.6377, -527...."


In [192]:
before_data =list()
for cell_rec in doses_df['Before IV'].values:
    for rec in abf_recordings_df.index:
        if str(cell_rec) in rec:
            before_data.append(abf_recordings_df.loc[rec,'K_Currents'])
doses_df['Before'] = before_data


after_data =list()
for cell_rec in doses_df['After IV'].values:
    for rec in abf_recordings_df.index:
        if str(cell_rec) in rec:
            after_data.append(abf_recordings_df.loc[rec,'K_Currents'])
doses_df['After'] = after_data

display(doses_df.head())

,Before IV,During,After IV,Before,After
Cell,,,,,
2023x12x13_SLICK_c001_HEK,2023x12x13_SLICK_c001_HEK_0002,3,2023x12x13_SLICK_c001_HEK_0004,"{'Ik': [-494.0796, -458.6792, -399.4751, -369....","{'Ik': [-772.7051, -707.70264, -603.02734, -51..."
2023x12x13_SLICK_c002_HEK,2023x12x13_SLICK_c002_HEK_0004,6,2023x12x13_SLICK_c002_HEK_0008,"{'Ik': [-72.32666, -67.74902, -54.93164, -48.8...","{'Ik': [-194.39697, -160.52246, -156.55518, -1..."
2023x12x14_SLICK_c001_HEK,2023x12x14_SLICK_c001_HEK_0000,2,2023x12x14_SLICK_c001_HEK_0003,"{'Ik': [38.45215, 26.245117, 12.512207, 18.005...","{'Ik': [44.86084, 22.888184, 14.343262, -4.272..."
2023x12x15_SLICK_c001_HEK,2023x12x15_SLICK_c001_HEK_0003,4,2023x12x15_SLICK_c001_HEK_0005,"{'Ik': [-589.5996, -513.0005, -450.74463, -388...","{'Ik': [-379.02832, -353.69873, -306.39648, -2..."
2023x12x15_SLICK_c002_HEK,2023x12x15_SLICK_c002_HEK_0002,3,2023x12x15_SLICK_c002_HEK_0004,"{'Ik': [-2253.418, -2008.3618, -1755.9814, -14...","{'Ik': [-1978.7598, -1765.1367, -1549.6826, -1..."


In [205]:
for r in doses_df.index:
    before = doses_df.loc[r,'Before']
    max_I = np.max(before['Ik_corr'])
    before['Ik_norm'] = before['Ik_corr'] / max_I
    doses_df.at[r,'Before'] = before

    after = doses_df.loc[r,'After']
    after['Ik_norm'] = after['Ik_corr'] / max_I
    doses_df.at[r,'After'] = after

    common = sorted(list(set(after['Vm']).intersection(set(before['Vm']))))

    delta = {'Vm':common,'Ikb':list(),'Ika':list()}

    for i,v in enumerate(before['Vm']):
        if v in delta['Vm']:
            delta['Ikb'].append( before['Ik_norm'][i] )

    for i,v in enumerate(after['Vm']):
        if v in delta['Vm']:
            delta['Ika'].append( after['Ik_norm'][i] )
    doses_df.at[r,'delta'] = delta

doses_df

,Before IV,During,After IV,Before,After,delta
Cell,,,,,,
2023x12x13_SLICK_c001_HEK,2023x12x13_SLICK_c001_HEK_0002,3,2023x12x13_SLICK_c001_HEK_0004,"{'Ik': [-494.0796, -458.6792, -399.4751, -369....","{'Ik': [-772.7051, -707.70264, -603.02734, -51...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x13_SLICK_c002_HEK,2023x12x13_SLICK_c002_HEK_0004,6,2023x12x13_SLICK_c002_HEK_0008,"{'Ik': [-72.32666, -67.74902, -54.93164, -48.8...","{'Ik': [-194.39697, -160.52246, -156.55518, -1...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x14_SLICK_c001_HEK,2023x12x14_SLICK_c001_HEK_0000,2,2023x12x14_SLICK_c001_HEK_0003,"{'Ik': [38.45215, 26.245117, 12.512207, 18.005...","{'Ik': [44.86084, 22.888184, 14.343262, -4.272...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c001_HEK,2023x12x15_SLICK_c001_HEK_0003,4,2023x12x15_SLICK_c001_HEK_0005,"{'Ik': [-589.5996, -513.0005, -450.74463, -388...","{'Ik': [-379.02832, -353.69873, -306.39648, -2...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c002_HEK,2023x12x15_SLICK_c002_HEK_0002,3,2023x12x15_SLICK_c002_HEK_0004,"{'Ik': [-2253.418, -2008.3618, -1755.9814, -14...","{'Ik': [-1978.7598, -1765.1367, -1549.6826, -1...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c003_HEK,2023x12x15_SLICK_c003_HEK_0001,2,2023x12x15_SLICK_c003_HEK_0003,"{'Ik': [-2244.5679, -2047.7295, -1866.4551, -1...","{'Ik': [-1760.8643, -1618.042, -1467.8955, -13...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c005_HEK,2023x12x15_SLICK_c005_HEK_0002,3,2023x12x15_SLICK_c005_HEK_0005,"{'Ik': [-21.66748, -3.9672852, -43.029785, -36...","{'Ik': [-64.086914, 73.85254, -59.814453, 0.30...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c006_HEK,2023x12x15_SLICK_c006_HEK_0002,4,2023x12x15_SLICK_c006_HEK_0007,"{'Ik': [0.0, 32.958984, -2.4414062, -16.174316...","{'Ik': [-3.6621094, -15.563965, 2.4414062, 5.4...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c007_HEK,2023x12x15_SLICK_c007_HEK_0001,3,2023x12x15_SLICK_c007_HEK_0005,"{'Ik': [1.8310547, 39.0625, 38.45215, 10.98632...","{'Ik': [10.986328, 11.901855, 3.3569336, -2.13...","{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."


In [221]:
c = doses_df.index[0]
a = pd.DataFrame(doses_df.loc[c,'delta']).set_index('Vm')['Ika']
a

Vm
-120   -1.742448
-110   -1.562003
-100   -0.864865
-90    -0.362480
-80    -0.241653
-70    -0.410970
-60    -0.047695
-50    -0.002385
-40     0.042925
-30    -0.086645
-20     0.054054
-10    -0.007949
 0      0.033386
 10     0.034976
 20     0.048490
 30     0.181240
 40     0.151033
 50     0.351351
Name: Ika, dtype: float64

In [216]:
for c in doses_df.index:
    a = pd.DataFrame(doses_df.loc[c,'delta'])


,delta
Cell,
2023x12x13_SLICK_c001_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x13_SLICK_c002_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x14_SLICK_c001_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c001_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c002_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c003_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c005_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c006_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
2023x12x15_SLICK_c007_HEK,"{'Vm': [-120, -110, -100, -90, -80, -70, -60, ..."
